<a href="https://colab.research.google.com/github/eugeniaring/Medium-Articles/blob/main/advanced_RAG_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries

In [ ]:
! pip install wikipedia
! pip install llama-index

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=580abd11a8cf8afcb14382c3b0d487b9255a6d404a842f0351381185bea31383
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Import libraries

In [ ]:
import logging
import sys
import os
from google.colab import userdata

import nest_asyncio
from llama_index.core import (
    VectorStoreIndex,
    ServiceContext,
    download_loader,
)
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType
from llama_index.core.indices.query.query_transform.base import StepDecomposeQueryTransform
from llama_index.llms.openai import OpenAI
from llama_index.core.query_engine import SubQuestionQueryEngine, RouterQueryEngine, MultiStepQueryEngine
from llama_index.core.selectors.pydantic_selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.response.notebook_utils import display_source_node

## Extract data from Wikipedia

In [ ]:
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()
pages = ['Emma_Stone', 'La_La_Land', 'Ryan_Gosling']
documents = loader.load_data(pages=pages, auto_suggest=False, redirect = False)

<ipython-input-3-fbaf380908f0>:1: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  WikipediaReader = download_loader("WikipediaReader")


## Set up openai and huggingface api token

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

In [ ]:
gpt3 = OpenAI(temperature=0, model="gpt-3.5-turbo-0125")

## indexing phase

In [ ]:
embed_model = OpenAIEmbedding(model= OpenAIEmbeddingModelType.TEXT_EMBED_ADA_002)
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)
index = VectorStoreIndex.from_documents(documents, service_context=service_context_gpt3)

<ipython-input-6-f80b0a74bb4d>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, chunk_size = 256, chunk_overlap=0, embed_model=embed_model)


## retrieval phase

In [ ]:
retriever = index.as_retriever(similarity_top_k=3)

In [ ]:
# The response from original prompt
from llama_index.core.prompts import PromptTemplate

template = (
"We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
    "Don't give an answer unless it is supported by the context above.\n"
)

qa_template = PromptTemplate(template)

## generation phase

### Query 1

In [ ]:
question = "What is the plot of the film that made Emma Stone win her first Academy Award?"

contexts = retriever.retrieve(question)

context_list = [n.get_content() for n in contexts]

prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

response = gpt3.complete(prompt)
print(str(response))

The plot of the film that made Emma Stone win her first Academy Award is not explicitly mentioned in the provided context.


### Query 2

In [ ]:
question = "Compare the families of Emma Stone and Ryan Gosling"

contexts = retriever.retrieve(question)

context_list = [n.get_content() for n in contexts]

prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

response = gpt3.complete(prompt)
print(str(response))

Based on the context provided, there is no specific information about the families of Emma Stone and Ryan Gosling. The information mainly focuses on their careers, experiences in the film industry, and their collaboration on various projects. Therefore, it is not possible to compare the families of Emma Stone and Ryan Gosling based on the given information.


### Query 3

In [ ]:
question = "Compare the career beggings of Emma Stone and Ryan Gosling"

contexts = retriever.retrieve(question)

context_list = [n.get_content() for n in contexts]

prompt = qa_template.format(context_str="\n\n".join(context_list), query_str=question)

response = gpt3.complete(prompt)
print(str(response))

Both Emma Stone and Ryan Gosling began their careers at a young age, with Stone starting in theater productions as a child and Gosling starting in television as a teenager. Stone made her television debut in a reality show that produced only an unsold pilot, while Gosling had small television roles before transitioning to film. Stone appeared in well-received teen comedy films like Superbad, Zombieland, and Easy A, while Gosling had roles in various films before being cast in La La Land. Both actors drew from their own experiences as aspiring artists for their roles in the film, with Stone struggling to get auditions and Gosling experiencing a casting director taking a phone call during his emotional audition. Stone's love for musicals and dream of bursting into song influenced her performance in La La Land, while Gosling's decision to turn down a role in Beauty and the Beast in favor of La La Land showcases his commitment to challenging and diverse roles. Overall, both Stone and Gosli

## (1) Basic strategies

In [ ]:
service_context_gpt3 = ServiceContext.from_defaults(llm=gpt3, chunk_size = 512, chunk_overlap=50, embed_model=embed_model)

index = VectorStoreIndex.from_documents(
    documents, service_context=service_context_gpt3
)

query_engine = index.as_query_engine(similarity_top_k=4)

In [ ]:
response = query_engine.query("What is the plot of the film that made Emma Stone win her first Academy Award?")
print(response)

The film that made Emma Stone win her first Academy Award is a romantic musical called La La Land.


In [ ]:
response = query_engine.query("Compare the families of Emma Stone and Ryan Gosling")
print(response)

Emma Stone has expressed her close relationship with her family and mentioned being blessed with great family and people around her. She has also shared about her mother's battle with breast cancer and their celebration by getting matching tattoos. On the other hand, there is no specific information provided about Ryan Gosling's family or his personal relationships in the context.


## (2) Reranking

### Before reranking

In [ ]:
retriever = index.as_retriever(similarity_top_k=4)
query = "Compare the families of Emma Stone and Ryan Gosling"
nodes = retriever.retrieve(query)
for node in nodes:
    print('----------------------------------------------------')
    display_source_node(node, source_length = 500)

----------------------------------------------------


**Node ID:** 9b3817fe-3a3f-4417-83d2-2e2996c8b468<br>**Similarity:** 0.8415899563985404<br>**Text:** Emily Jean "Emma" Stone (born November 6, 1988) is an American actress and producer. She is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, and two Golden Globe Awards. In 2017, she was the world's highest-paid actress and named by Time magazine as one of the 100 most influential people in the world.
Born and raised in Scottsdale, Arizona, Stone began acting as a child in a theater production of The Wind in the Willows in 2000. As a teenager,...<br>

----------------------------------------------------


**Node ID:** 1bef0308-8b0f-4f7e-9cd6-92ce5acf811f<br>**Similarity:** 0.831147173341674<br>**Text:** Coincidentally, Gosling turned down the Beast role in Beauty and the Beast in favor of La La Land. Chazelle subsequently decided to make his characters somewhat older, with experience in struggling to make their dreams, rather than younger newcomers just arriving in Los Angeles.Emma Stone plays Mia, an aspiring actress in Los Angeles. Stone has loved musicals since she saw Les Misérables when she was eight years old. She said "bursting into song has always been a real dream of mine", and her ...<br>

----------------------------------------------------


**Node ID:** 576ae445-b12e-4d20-99b7-5e5a91ee7d74<br>**Similarity:** 0.8289486590392277<br>**Text:** Stone was named the best-dressed woman of 2012 by Vogue and was included on similar listings by Glamour in 2013 and 2015, and People in 2014.


== Personal life ==
Stone moved from Los Angeles to Greenwich Village, New York, in 2009. In 2016, she moved back to Los Angeles. Despite significant media attention, she refuses to publicly discuss her personal life. Concerned with living a normal life, Stone has said she dislikes receiving paparazzi attention outside her home. She has expressed her ...<br>

----------------------------------------------------


**Node ID:** 093bc5c2-af9b-4f2a-aaf7-73b63237463c<br>**Similarity:** 0.8228748702164247<br>**Text:** == Notes ==


== References ==


== Literary sources ==
Hollinger, Karen (2022). "Amy Adams and Emma Stone: Escaping the Ingénue". In Rybin, Steven (ed.). Stellar Transformations: Movie Stars of the 2010s. Rutgers University Press. pp. 34–52. ISBN 978-1-978818-33-0.
Owings, Lisa (2014). Emma Stone: Breakout Movie Star. ABDO Publishing Company. p. 22; 33. ISBN 978-1-62968-026-2.
Schuman, Michael A. (2013). Emma!: Amazing Actress Emma Stone. Enslow Publishers. ISBN 978-0-7660-4113-4.


== Exter...<br>

### FlagEmbeddingReranker

In [ ]:
%pip install llama-index-postprocessor-flag-embedding-reranker
!pip install git+https://github.com/FlagOpen/FlagEmbedding.git

  Cloning https://github.com/FlagOpen/FlagEmbedding.git to /tmp/pip-req-build-r863im0k
  Running command git clone --filter=blob:none --quiet https://github.com/FlagOpen/FlagEmbedding.git /tmp/pip-req-build-r863im0k
  Resolved https://github.com/FlagOpen/FlagEmbedding.git to commit 0fe98ddefbd9d2caced52e94b9f8421d5ba7317f
  Preparing metadata (setup.py) ... done


In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)
from llama_index.core.schema import QueryBundle

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')
os.environ['HF_TOKEN'] = HF_TOKEN

In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import (
    FlagEmbeddingReranker,
)
from llama_index.core.schema import QueryBundle

reranker = FlagEmbeddingReranker(
    top_n = 4,
    model = "BAAI/bge-reranker-base",
)

query_bundle = QueryBundle(query_str=query)
ranked_nodes = reranker._postprocess_nodes(nodes, query_bundle = query_bundle)
for ranked_node in ranked_nodes:
    print('----------------------------------------------------')
    display_source_node(ranked_node, source_length = 500)

----------------------------------------------------


**Node ID:** 9b3817fe-3a3f-4417-83d2-2e2996c8b468<br>**Similarity:** 3.0143558979034424<br>**Text:** Emily Jean "Emma" Stone (born November 6, 1988) is an American actress and producer. She is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, and two Golden Globe Awards. In 2017, she was the world's highest-paid actress and named by Time magazine as one of the 100 most influential people in the world.
Born and raised in Scottsdale, Arizona, Stone began acting as a child in a theater production of The Wind in the Willows in 2000. As a teenager,...<br>

----------------------------------------------------


**Node ID:** 576ae445-b12e-4d20-99b7-5e5a91ee7d74<br>**Similarity:** 2.2117154598236084<br>**Text:** Stone was named the best-dressed woman of 2012 by Vogue and was included on similar listings by Glamour in 2013 and 2015, and People in 2014.


== Personal life ==
Stone moved from Los Angeles to Greenwich Village, New York, in 2009. In 2016, she moved back to Los Angeles. Despite significant media attention, she refuses to publicly discuss her personal life. Concerned with living a normal life, Stone has said she dislikes receiving paparazzi attention outside her home. She has expressed her ...<br>

----------------------------------------------------


**Node ID:** 1bef0308-8b0f-4f7e-9cd6-92ce5acf811f<br>**Similarity:** 1.6185210943222046<br>**Text:** Coincidentally, Gosling turned down the Beast role in Beauty and the Beast in favor of La La Land. Chazelle subsequently decided to make his characters somewhat older, with experience in struggling to make their dreams, rather than younger newcomers just arriving in Los Angeles.Emma Stone plays Mia, an aspiring actress in Los Angeles. Stone has loved musicals since she saw Les Misérables when she was eight years old. She said "bursting into song has always been a real dream of mine", and her ...<br>

----------------------------------------------------


**Node ID:** 093bc5c2-af9b-4f2a-aaf7-73b63237463c<br>**Similarity:** -5.185266971588135<br>**Text:** == Notes ==


== References ==


== Literary sources ==
Hollinger, Karen (2022). "Amy Adams and Emma Stone: Escaping the Ingénue". In Rybin, Steven (ed.). Stellar Transformations: Movie Stars of the 2010s. Rutgers University Press. pp. 34–52. ISBN 978-1-978818-33-0.
Owings, Lisa (2014). Emma Stone: Breakout Movie Star. ABDO Publishing Company. p. 22; 33. ISBN 978-1-62968-026-2.
Schuman, Michael A. (2013). Emma!: Amazing Actress Emma Stone. Enslow Publishers. ISBN 978-0-7660-4113-4.


== Exter...<br>

In [ ]:
query_engine = index.as_query_engine(       # add reranker to query_engine
    similarity_top_k = 4,
    node_postprocessors=[reranker]
)
response = query_engine.query("Compare the families of Emma Stone and Ryan Gosling")
print(response)

Both Emma Stone and Ryan Gosling have close relationships with their families. Stone has expressed her gratitude for having a great family and people around her who keep her grounded. Gosling, on the other hand, has drawn from his own experiences as an aspiring artist, indicating a connection to his personal background.


Both Emma Stone and Ryan Gosling have close relationships with their families. Stone has expressed her gratitude for having a great family and people around her who keep her grounded. Gosling, on the other hand, has drawn from his own experiences as an aspiring artist, indicating a connection to his personal background.


### RankGPTRerank

In [ ]:
%pip install llama-index-postprocessor-rankgpt-rerank

In [ ]:
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

reranker = RankGPTRerank(
    top_n = 4,
    llm = OpenAI(model="gpt-3.5-turbo-0125"),
)

In [ ]:
query_bundle = QueryBundle(query_str=query)
ranked_nodes = reranker._postprocess_nodes(nodes, query_bundle = query_bundle)
for ranked_node in ranked_nodes:
    print('----------------------------------------------------')
    display_source_node(ranked_node, source_length = 500)

----------------------------------------------------


**Node ID:** 1bef0308-8b0f-4f7e-9cd6-92ce5acf811f<br>**Similarity:** 1.6185210943222046<br>**Text:** Coincidentally, Gosling turned down the Beast role in Beauty and the Beast in favor of La La Land. Chazelle subsequently decided to make his characters somewhat older, with experience in struggling to make their dreams, rather than younger newcomers just arriving in Los Angeles.Emma Stone plays Mia, an aspiring actress in Los Angeles. Stone has loved musicals since she saw Les Misérables when she was eight years old. She said "bursting into song has always been a real dream of mine", and her ...<br>

----------------------------------------------------


**Node ID:** 9b3817fe-3a3f-4417-83d2-2e2996c8b468<br>**Similarity:** 3.0143558979034424<br>**Text:** Emily Jean "Emma" Stone (born November 6, 1988) is an American actress and producer. She is the recipient of various accolades, including two Academy Awards, two British Academy Film Awards, and two Golden Globe Awards. In 2017, she was the world's highest-paid actress and named by Time magazine as one of the 100 most influential people in the world.
Born and raised in Scottsdale, Arizona, Stone began acting as a child in a theater production of The Wind in the Willows in 2000. As a teenager,...<br>

----------------------------------------------------


**Node ID:** 576ae445-b12e-4d20-99b7-5e5a91ee7d74<br>**Similarity:** 2.2117154598236084<br>**Text:** Stone was named the best-dressed woman of 2012 by Vogue and was included on similar listings by Glamour in 2013 and 2015, and People in 2014.


== Personal life ==
Stone moved from Los Angeles to Greenwich Village, New York, in 2009. In 2016, she moved back to Los Angeles. Despite significant media attention, she refuses to publicly discuss her personal life. Concerned with living a normal life, Stone has said she dislikes receiving paparazzi attention outside her home. She has expressed her ...<br>

----------------------------------------------------


**Node ID:** 093bc5c2-af9b-4f2a-aaf7-73b63237463c<br>**Similarity:** -5.185266971588135<br>**Text:** == Notes ==


== References ==


== Literary sources ==
Hollinger, Karen (2022). "Amy Adams and Emma Stone: Escaping the Ingénue". In Rybin, Steven (ed.). Stellar Transformations: Movie Stars of the 2010s. Rutgers University Press. pp. 34–52. ISBN 978-1-978818-33-0.
Owings, Lisa (2014). Emma Stone: Breakout Movie Star. ABDO Publishing Company. p. 22; 33. ISBN 978-1-62968-026-2.
Schuman, Michael A. (2013). Emma!: Amazing Actress Emma Stone. Enslow Publishers. ISBN 978-0-7660-4113-4.


== Exter...<br>

In [ ]:
query_engine = index.as_query_engine(       # add reranker to query_engine
    similarity_top_k = 4,
    node_postprocessors=[reranker]
)
response = query_engine.query("Compare the families of Emma Stone and Ryan Gosling")
print(response)

Both Emma Stone and Ryan Gosling have close relationships with their families. Stone has expressed gratitude for having a great family and supportive people around her, while Gosling has drawn from his own experiences as an aspiring artist, indicating a connection to his family as well. Stone's family has played a significant role in her life, supporting her career pursuits and celebrating important milestones with her. Similarly, Gosling's experiences as an actor have likely been influenced by his family dynamics and relationships.


Both Emma Stone and Ryan Gosling have close relationships with their families. Stone has expressed gratitude for having a great family and supportive people around her, while Gosling has drawn from his own experiences as an aspiring artist, indicating a connection to his family as well. Stone's family has played a significant role in her life, supporting her career pursuits and celebrating important milestones with her. Similarly, Gosling's experiences as an actor have likely been influenced by his family dynamics and relationships.



## (3) Querying Transformations

### HYDE

In [ ]:
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine.transform_query_engine import (
    TransformQueryEngine,
)

index = VectorStoreIndex.from_documents(documents, service_context=service_context_gpt3)
query_engine = index.as_query_engine(similarity_top_k=4)

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [ ]:
response = hyde_query_engine.query("Compare the families of Emma Stone and Ryan Gosling")

In [ ]:
print(response)

Both Emma Stone and Ryan Gosling come from diverse family backgrounds. Ryan Gosling's parents were of part French Canadian descent, along with some German, English, Scottish, and Irish. He grew up in a family that practiced the Church of Jesus Christ of Latter-day Saints. On the other hand, Emma Stone was born and raised in Scottsdale, Arizona, without specific details provided about her family's ethnic background or religious affiliation.


Both Emma Stone and Ryan Gosling come from diverse family backgrounds. Ryan Gosling's parents were of part French Canadian descent, along with some German, English, Scottish, and Irish. He grew up in a family that practiced the Church of Jesus Christ of Latter-day Saints. On the other hand, Emma Stone was born and raised in Scottsdale, Arizona, without specific details provided about her family's ethnic background or religious affiliation.


### Multi-step Query Transformation

In [ ]:
step_decompose_transform_gpt3 = StepDecomposeQueryTransform(gpt3, verbose=True)
index_summary = "Breaks down the initial query"
multi_step_query_engine = MultiStepQueryEngine(
    query_engine=query_engine,
    query_transform=step_decompose_transform_gpt3,
    index_summary=index_summary
)

In [ ]:
response = multi_step_query_engine.query("Compare the families of Emma Stone and Ryan Gosling")

> Current query: Compare the families of Emma Stone and Ryan Gosling
> New query: What is the family background of Emma Stone and Ryan Gosling?
> Current query: Compare the families of Emma Stone and Ryan Gosling
> New query: What is the specific family background of Emma Stone?
> Current query: Compare the families of Emma Stone and Ryan Gosling
> New query: What is Emma Stone's relationship with her family like?


In [ ]:
print(response)

Both Ryan Gosling and Emma Stone have strong family connections, with Emma Stone emphasizing the importance of her family in keeping her grounded. Ryan Gosling's family background includes a mix of French Canadian, German, English, Scottish, and Irish heritage.


Both Ryan Gosling and Emma Stone have strong family connections, with Emma Stone emphasizing the importance of her family in keeping her grounded. Ryan Gosling's family background includes a mix of French Canadian, German, English, Scottish, and Irish heritage.
